# Answering Business Questions Using SQL

## Introduction 
In this project, we're going to practice using SQL to answer business questions.

We'll use the Chinook database. The Chinook database is provided as a SQLite database file called [chinook.db](https://github.com/lerocha/chinook-database). A copy of the database schema is below:
![db schema](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)
## Overview 

We'll use the following code to connect our Jupyter Notebook to our database file:


In [6]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [13]:
# Import Libraries
import pandas as pd
import numpy as np
import sqlite3
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Let's start by getting familiar with our data. 

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [5]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting Albums to Purchase
The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| Artist Name          | Genre   |
|----------------------|---------|
| Regal                | Hip-Hop |
| Red Tone             | Punk    |
| Meteor and the Girls | Pop     |
| Slim Jim Bites       | Blues   |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [8]:
%%sql
WITH usa_tracks_sold AS
                           (
                            SELECT il.* FROM invoice_line il
                            INNER JOIN invoice i on il.invoice_id = i.invoice_id
                            INNER JOIN customer c on c.customer_id = i.customer_id
                            WHERE c.country = "USA"
                           )

    
   
SELECT
       g.name genre,
       count(uts.invoice_line_id) tracks_sold,
       ROUND(CAST(count(uts.invoice_line_id) AS FLOAT) * 100.00 / (
                                                    SELECT COUNT(*) 
                                                      FROM usa_tracks_sold
                                                    ), 0) AS percentage_sold

  FROM usa_tracks_sold AS uts
 INNER JOIN track t on t.track_id = uts.track_id
 INNER JOIN genre g on g.genre_id = t.genre_id
 GROUP BY 1
 ORDER BY 2 DESC
 LIMIT 10;

Done.


genre,tracks_sold,percentage_sold
Rock,561,53.0
Alternative & Punk,130,12.0
Metal,124,12.0
R&B/Soul,53,5.0
Blues,36,3.0
Alternative,35,3.0
Latin,22,2.0
Pop,22,2.0
Hip Hop/Rap,20,2.0
Jazz,14,1.0


In [10]:
query = """ 
WITH usa_tracks_sold AS
                           (
                            SELECT il.* FROM invoice_line il
                            INNER JOIN invoice i on il.invoice_id = i.invoice_id
                            INNER JOIN customer c on c.customer_id = i.customer_id
                            WHERE c.country = "USA"
                           )

    
   
SELECT
       g.name genre,
       count(uts.invoice_line_id) tracks_sold,
       ROUND(CAST(count(uts.invoice_line_id) AS FLOAT) * 100.00 / (
                                                    SELECT COUNT(*) 
                                                      FROM usa_tracks_sold
                                                    ), 0) AS percentage_sold

  FROM usa_tracks_sold AS uts
 INNER JOIN track t on t.track_id = uts.track_id
 INNER JOIN genre g on g.genre_id = t.genre_id
 GROUP BY 1
 ORDER BY 2 DESC
 LIMIT 10;
 """

# SQL output to pandas:

conn = sqlite3.connect("chinook.db")
df = pd.read_sql_query(query, conn) 
df = df.set_index("genre", drop=True) # removing index

df

,tracks_sold,percentage_sold
genre,,
Rock,561,53.0
Alternative & Punk,130,12.0
Metal,124,12.0
R&B/Soul,53,5.0
Blues,36,3.0
Alternative,35,3.0
Latin,22,2.0
Pop,22,2.0
Hip Hop/Rap,20,2.0


From our output, we can see that most likely the top 5 genres that sell the best in the USA is:

* Rock
* Alternative & Punk
* Metal
* R&B/Soul
* Blues

From that knowledge, we can recommend "Red Tone" artist to be the most reasonable one for promotion. If there is enough money for more than one artist, then the second choice is "Slim Jim Bites". Third best choice is "Meteor and the Girls".

## Analyzing Employee Sales Performance
Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

We will consider whether any extra columns from the employee table explain any variance we see, or whether the variance might instead be indicative of employee performance.


In [17]:
%%sql
SELECT *
 FROM employee

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2016-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2017-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2017-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [18]:
%%sql

WITH invoice_total AS
                                 (
                                 SELECT 
                                        SUM(i.total) AS total_purchase,
                                        i.customer_id
                                   FROM invoice i
                                  GROUP BY 2
                                 ),
    customer_invoice_total AS
                                (
                                SELECT
                                       i.customer_id,
                                       i.total_purchase,
                                       c.support_rep_id,
                                       c.country AS customer_country
                                  FROM customer AS c
                                 INNER JOIN invoice_total AS i ON i.customer_id = c.customer_id
                                ),
            employee_cit AS
                                (
                                SELECT
                                       e.first_name ||" "|| e.last_name AS employee_name,
                                       ROUND(cit.total_purchase, 2) || " $" AS total_purchase,
                                       e.country AS employee_country,
                                       cit.customer_country,
                                       (CASE
                                            WHEN cit.customer_country = 'Canada' THEN 'The same'
                                        END) AS employee_vs_cust_country_same,
                                       (CASE
                                            WHEN cit.customer_country <> 'Canada' THEN 'Different country'
                                        END) AS employee_vs_cust_country_dif
                                  FROM employee AS e
                                 INNER JOIN customer_invoice_total AS cit ON cit.support_rep_id = e.employee_id
                                ),
            sum_country AS
                               (
                               SELECT
                                      ecit.employee_name,
                                      SUM(
                                          CASE WHEN ecit.employee_vs_cust_country_same = 'The same'
                                          THEN 1
                                          ELSE 0
                                          END
                                          ) AS customer_canada,
                                      SUM(
                                          CASE WHEN ecit.employee_vs_cust_country_dif = 'Different country'
                                          THEN 1 
                                          ELSE 0 
                                          END
                                          ) AS customer_dif_country
                                 FROM employee_cit AS ecit
                                GROUP BY 1
                               ),
        employee_income AS
                               (
                               SELECT 
                                      ecit.employee_name,
                                      ROUND(SUM(ecit.total_purchase), 2) ||" $"  AS total_purchase     
                                 FROM employee_cit AS ecit
                                GROUP BY 1
                                ORDER BY 2 DESC
                               )
SELECT
       ei.employee_name,
       ei.total_purchase,
       sc.customer_canada,
       sc.customer_dif_country
  FROM employee_income AS ei
 INNER JOIN sum_country AS sc ON sc.employee_name = ei.employee_name

Done.


employee_name,total_purchase,customer_canada,customer_dif_country
Jane Peacock,1731.51 $,5,16
Margaret Park,1584.0 $,1,19
Steve Johnson,1393.92 $,2,16


As we can see Jane Peacock is the best salesman. In the second place is Margaret Park. Because we had realized that there are some extra data that can help us understand "the bigger picture" better, let's analyze them now.

We can see that Jane Peacock is the best, but more in the "inside market". To be more specific: in Canada. On the other hand, Margaret Park is more effective with clients outside the border. It makes those two employees a team of the best inside and outside country's salesmen.

This knowledge can be used for planning the further development of the company.

The last thing worth mentioning here is, that the total number of clients for each salesman is slightly different. Jane Peacock has them the most: 21. Margaret Park has 20. Steve Johnson has the smallest amount of clients: 18. That can explain the lower income made by him.

When we calculate what's the average of income from one client in the case of Jane Peacock we will get:

1731.51 / 21 = 82,45$

If Steve Johnson had 21 clients then he would earn more. Steve has 18 clients, so when we add 3 more (so we could compare him to Jane) with assumption that each of those clients will bring 82,45$ then the income of Steve would be: 

1393,92 * 247,36 = 1641,28 $

Jane worked out 1731.51 $.

With 20 clients (comparing Steve to `Margaret) that would be:

1393,92 * 164,91 = 1558,83 $

Margaret worked out: 1584 $.

The conclusion is, probably Steve Johnson is a great salesman too, but he just has fewer customers. We recommend checking - if it's possible - how his performance would be like with a bigger client's database - if this is the case. If employees are responsible for finding clients, or persuade someone to be one, then, it points out that indeed Steve Johnson has lower performance and brings lower income for the company.

## Analyzing Sales by Country

Our next task is to analyze the sales data for customers from each different country. We have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, we have been directed to calculate data, for each country, on the:

total number of customers
total value of sales
average value of sales per customer
average order value
Because there are a number of countries with only one customer, we should group these customers as "Other" in our analysis. We can use the following 'trick' to force the ordering of "Other" to last in our analysis.

If there is a particular value that we would like to force to the top or bottom of results, we can put what would normally be our most outer query in a subquery with a case statement that adds a numeric column, and then in the outer query sort by that column. Here's an example - let's start by creating a view so we're working with a manageable number of rows:

In [23]:
%%sql
CREATE VIEW top_5_names AS
     SELECT
         first_name,
         count(customer_id) count
     FROM customer
     GROUP by 1
     ORDER by 2 DESC
     LIMIT 5;

SELECT * FROM top_5_names;


Done.
Done.


first_name,count
Frank,2
Mark,2
Aaron,1
Alexandre,1
Astrid,1


Next, inside a subquery, we'll select all values from our view and add a sorting column using a case statement, before sorting using that new column in the outer query.

In [24]:
%%sql
SELECT
    first_name,
    count
FROM
    (
    SELECT
        t5.*,
        CASE
            WHEN t5.first_name = "Mark" THEN 1
            ELSE 0
        END AS sort
    FROM top_5_names t5
   )
ORDER BY sort ASC

Done.


first_name,count
Frank,2
Aaron,1
Alexandre,1
Astrid,1
Mark,2


We should be able to adapt this technique into our query to force 'Other' to the bottom of our results. When working through this exercise, we will need multiple subqueries and joins, so it will be easy to read and understood by our colleagues.

Next we will:

* Write a query that collates data on purchases from different countries.
    * Where a country has only one customer, collect them into an "Other" group.
    * The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.
    * For each country, include:
        * total number of customers
        * total value of sales
        * average value of sales per customer
        * average order value (unit price)

In [25]:
query = """
 
WITH customer_total_trancaction AS
                                    (
                                     SELECT
                                            c.country,
                                            i.customer_id,
                                            COUNT(i.invoice_id) AS sales_per_customer,
                                            SUM(i.total) AS total_transaction,
                                            ROUND(SUM(i.total), 2) AS total_income_from_transaction
                                       FROM invoice AS i
                                      INNER JOIN customer AS c ON c.customer_id = i.customer_id
                                      GROUP BY 1, 2
                                    ),
    country_customer_total_tran AS 
                                    (
                                     SELECT
                                            c.country,
                                            COUNT(ctt.customer_id) AS total_number_customers,
                                            ROUND(SUM(ctt.total_transaction), 2) || " $" AS total_sales,
                                            ROUND(AVG(ctt.sales_per_customer), 2) || " $" AS sales_per_customer,
                                            ROUND(AVG(ctt.total_transaction), 2) || " $" AS avg_order
                                       FROM customer AS c
                                      INNER JOIN customer_total_trancaction AS ctt ON ctt.customer_id = c.customer_id
                                      GROUP BY 1
                                    ),
                                
        
             country_customers AS
                                    (
                                     SELECT * 
                                        FROM country_customer_total_tran 
                                       WHERE total_number_customers > 1
                                       ORDER BY 1
                                    ),
    
           country_other_customer AS
                                      (
                                        SELECT 
                                               COUNT(cctt.country) AS country,
                                               COUNT(cctt.total_number_customers) AS total_number_customers,
                                               SUM(cctt.total_sales) AS total_sales,
                                               AVG(cctt.sales_per_customer) AS sales_per_customer,
                                               AVG(cctt.avg_order) AS avg_order
                                          FROM country_customer_total_tran AS cctt
                                         WHERE total_number_customers = 1 
                                      ),
            
                 other_countries AS   (SELECT CASE 
                                              WHEN coc.country = (SELECT coc.country 
                                                                FROM country_other_customer)       
                                                                THEN "Other"
                                               END AS country, 
                                                      coc.total_number_customers,
                                                      coc.total_sales || " $",
                                                      coc.sales_per_customer || " $",
                                                      ROUND(coc.avg_order, 2) || " $"
                                              FROM country_other_customer AS coc
                                        ),
                  united_all AS         (
                                         SELECT * FROM country_customers
                                          UNION
                                         SELECT * FROM other_countries
                                        )
                 
SELECT country,
       total_number_customers,
       total_sales,
       ROUND(total_number_customers * 100.0/(SELECT SUM(total_number_customers) 
                                               FROM united_all), 2) pct_customers,
       ROUND(total_sales * 100.0/(SELECT SUM(total_sales) 
                                    FROM united_all), 2) pct_sales,       
       sales_per_customer,
       avg_order
  FROM (SELECT united_all.*,
               CASE
               WHEN united_all.country = "Other" THEN 1
               ELSE 0
               END AS sort
          FROM united_all)
 ORDER BY sort ASC, 3 ASC;
 
 """
# SQL result to pandas df:
conn = sqlite3.connect("chinook.db")
df = pd.read_sql_query(query, conn).set_index("country", drop=True)
df

,total_number_customers,total_sales,pct_customers,pct_sales,sales_per_customer,avg_order
country,,,,,,
USA,13,1040.49 $,22.03,22.09,10.08 $,80.04 $
India,2,183.15 $,3.39,3.89,10.5 $,91.57 $
Portugal,2,185.13 $,3.39,3.93,14.5 $,92.56 $
United Kingdom,3,245.52 $,5.08,5.21,9.33 $,81.84 $
Czech Republic,2,273.24 $,3.39,5.80,15.0 $,136.62 $
Germany,4,334.62 $,6.78,7.11,10.25 $,83.66 $
France,5,389.07 $,8.47,8.26,10.0 $,77.81 $
Brazil,5,427.68 $,8.47,9.08,12.2 $,85.54 $
Canada,8,535.59 $,13.56,11.37,9.5 $,66.95 $


From our output, we can see that the most important countries with the greatest number of customers and total sales are:

1. USA
2. Canada
3. Brazil
4. France - the same number of customers as Brazil, but fewer sales.


## Albums vs Individual Tracks
The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

* purchase a whole album
* purchase a collection of one or more individual tracks.
The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

It is very common when we are performing an analysis to have 'edge cases' which prevent we from getting a 100% accurate answer to our question. In this instance, we have two edge cases to consider:

* Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
* Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.
In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.

In order to answer the question, we're going to have to identify whether each invoice has all the tracks from an album. We can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. We can find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track we pick, since if it's an album purchase, that album will be the same for all tracks.

To compare two tables of value, we can use the EXCEPT operator.

Let's say we had three tables in a database, as shown in the diagram below:
![diagramme1](https://s3.amazonaws.com/dq-content/191/test_tables.svg)

We want to find a way to compare the letters columns from test_table_2, and test_table_3 to test_table_1 to see if they they are identical to test_table_1. Let's use EXCEPT with the two identical tables and see what we get with the first two tables:
![diagramme2](https://s3.amazonaws.com/dq-content/191/compare_example_1.svg)

Now, let's compare what we get with test_table_1 and test_table_3:
![diagramme3](https://s3.amazonaws.com/dq-content/191/compare_example_2.svg)
If we run this directly in SQLite, we will get no result at all. This is useful to us - we can check whether the exception of two subqueries IS NULL. If we reverse the order of the tables around the EXCEPT operator we get the same thing.

Let's try reversing the order of the EXCEPT operator for the first two tables:
![diagramme4](https://s3.amazonaws.com/dq-content/191/compare_example_3.svg)
Here, we get a null value even though the two tables are not identical. That's because all of the values for letter in test_table_2 are also in test_table_1, even if test_table_1 has an extra value.

Because of this we need to combine both variations with AND clause:
![diagramme5](https://s3.amazonaws.com/dq-content/191/compare_example_4.svg)
Once we've made the comparison, we can  wrap it in a CASE statement to add a column that tells us if that invoice was an album purchase or not.

We have everything we need to collate data on album vs single track purchases. This is easily the hardest query we have written so far, so take our time, and remember the query writing tips from the first screen!

Next we'll do:

1. Write a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:
* Number of invoices
* Percentage of invoices
2. Write one to two sentences explaining our findings, and making a prospective recommendation on whether the Chinook store should continue to buy full albums from record companies

In [28]:
query = """
WITH invoice_album AS   
                    (
                     SELECT il.invoice_id,                                     
                            t.album_id
                       FROM invoice_line il
                       LEFT JOIN track t ON t.track_id = il.track_id
                      GROUP BY 1
                    ),

     album_vs_tracks AS 
                        (
                              SELECT ia.*,
                                CASE
                                WHEN 
                                     (
                                      SELECT il.track_id 
                                        FROM invoice_line il
                                       WHERE il.invoice_id = ia.invoice_id
                                      EXCEPT                                          
                                      SELECT t.track_id 
                                        FROM track t
                                       WHERE t.album_id = ia.album_id) IS NULL
                             
                                 AND 
                                     (
                                      SELECT t.track_id 
                                        FROM track t
                                       WHERE t.album_id = ia.album_id                                         
                                      EXCEPT                                          
                                      SELECT il.track_id 
                                        FROM invoice_line il
                                       WHERE il.invoice_id = ia.invoice_id
                                     ) IS NULL
                                   
                                THEN "Albums"
                                ELSE "Tracks"
                                END AS purchase                             
                           FROM invoice_album ia
                        )
                           
SELECT purchase,
       COUNT(*) AS num_invoices,
       ROUND(COUNT(*) * 100.0 /(SELECT COUNT(*)
                                  FROM invoice), 2) AS pct
  FROM album_vs_tracks
 GROUP BY 1
 ORDER BY 2 DESC;
 
 """
# SQL result to pandas df:
conn = sqlite3.connect("chinook.db")
df = pd.read_sql_query(query, conn).set_index("purchase", drop=True)
df

,num_invoices,pct
purchase,,
Tracks,500,81.43
Albums,114,18.57


As we mentioned before *" Management is currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album. "*

We can see, that the idea is correct. The company could save a lot of money. On the other hand, if all competitors do the same, then the one that won't stick to the trend will earn much more on albums. So it would be a "double-edged sword".

We recommend trying this new strategy, but at the same time, observing with caution what's going on in the market: what competitors sell.

Another approach is to check the situation even deeper: there might be albums worth being let alone.